## Install dependencies

In [2]:
# we have to enter OpenAI key here and then we can run all cells freely...
import os
from getpass import getpass
from openai import OpenAI

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
from openai import OpenAI

openai_client = OpenAI()
openai_client

## Downolad FAQ and create index and search function

In [4]:
# download parsed json FAQ
import json
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
# create a minsearch index from our FAQ 

from minsearch import AppendableIndex

index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)


In [6]:
# create a search function that uses our minsearch index to query FAQ

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
    )

    return results

# and test it - we get 5 top search results as we hardcoded num_results=5

search("How to set up Kafka") 


[{'text': 'In my set up, all of the dependencies listed in gradle.build were not installed in <project_name>-1.0-SNAPSHOT.jar.\nSolution:\nIn build.gradle file, I added the following at the end:\nshadowJar {\narchiveBaseName = "java-kafka-rides"\narchiveClassifier = \'\'\n}\nAnd then in the command line ran ‘gradle shadowjar’, and run the script from java-kafka-rides-1.0-SNAPSHOT.jar created by the shadowjar',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: <project_name>-1.0-SNAPSHOT.jar errors: package xxx does not exist even after gradle build',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'tip:As the videos have low audio so I downloaded them and used VLC media player with putting the audio to the max 200% of original audio and the audio became quite good or try to use auto caption generated on Youtube directly.\nKafka Python Videos - Rides.csv\nThere is no clear explanation of the rides.csv data that the producer.py python programs use. You can fin

In [7]:
# add a search_tool - a description for OpenAI SDK to let llm use 
# our search function - it is called << llm tool use >>
# if we don't describe the search function like below then OpenAI would 
# not be able to use it 

search_tool = {
    "type": "function",
    "name": "search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}


In [8]:
# this is how llm will call our search function:
    # f = globals()[f_name]
    # result = f(**args) - passing arguments to function which name we got from globals...
# it will use a call object and extract search function name 
# and arguments

def make_call(call):
    args = json.loads(call.arguments)
    f_name = call.name
    f = globals()[f_name]
    result = f(**args)
    result_json = json.dumps(result, indent=2)
    return {
        "type": "function_call_output",
        "call_id": call.call_id,
        "output": result_json,
    }



In [9]:
globals()["search"] 
# we should get internal system name of our search function:
# <function __main__.search(query)>

<function __main__.search(query)>

In [10]:
# and then we can do the pythonic trick - assign function name to a variable and 
# call it via variable as usual

f = globals()["search"]
f

<function __main__.search(query)>

In [11]:
# here is the trick - instead of calling 
# search("How to set up Kafka") we can simply do
f("How to set up Kafka") 
# check with similar search above!!

[{'text': 'In my set up, all of the dependencies listed in gradle.build were not installed in <project_name>-1.0-SNAPSHOT.jar.\nSolution:\nIn build.gradle file, I added the following at the end:\nshadowJar {\narchiveBaseName = "java-kafka-rides"\narchiveClassifier = \'\'\n}\nAnd then in the command line ran ‘gradle shadowjar’, and run the script from java-kafka-rides-1.0-SNAPSHOT.jar created by the shadowjar',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: <project_name>-1.0-SNAPSHOT.jar errors: package xxx does not exist even after gradle build',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'tip:As the videos have low audio so I downloaded them and used VLC media player with putting the audio to the max 200% of original audio and the audio became quite good or try to use auto caption generated on Youtube directly.\nKafka Python Videos - Rides.csv\nThere is no clear explanation of the rides.csv data that the producer.py python programs use. You can fin

## RAG internals - hard coded for better visibility

In [12]:
# LLM will need this developer prompt to learn how to use our tools

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

If you want to look up the answer, explain why before making the call. Use as many 
keywords from the user question as possible when making first requests.

Make multiple searches. Try to expand your search by using new keywords based on the results you
get from the search.

At the end, make a clarifying question based on what you presented and ask if there are 
other areas that the user wants to explore.
""".strip()



In [13]:
question = "I just discovered the course, can I join it now?"

In [14]:
chat_messages = [
    {"role": "developer", "content": developer_prompt},
    {"role": "user", "content": question}
]

# main piece of code is this - 
# response = openai_client.responses.create(
#         model='gpt-4o-mini',
#         input=chat_messages,
#         tools=[search_tool]
#     )
# - we use openai klient to talk with responses API and pass model name, tools and query + prompt

In [15]:
# now let llm decide if it need to search FAQ or can answer immediately
# after each loop iteration we increase our context with search results:
# - chat_messages.extend(response.output)

while True:
    response = openai_client.responses.create(
        model='gpt-4o-mini',
        input=chat_messages,
        tools=[search_tool]
    )
    
    chat_messages.extend(response.output)

    has_function_calls = False
    
    for entry in response.output:
        if entry.type == 'message':
            print(entry.content[0].text)
        if entry.type == 'function_call':
            print(entry)
            result = make_call(entry)
            chat_messages.append(result)
            has_function_calls = True

    if has_function_calls == False:
        break



ResponseFunctionToolCall(arguments='{"query":"join course late enrollment"}', call_id='call_HehINlpgVX9weBOFfjIjTb71', name='search', type='function_call', id='fc_68c1be65f55c81a0b3671035717eead909140a3903ee0d2c', status='completed')
ResponseFunctionToolCall(arguments='{"query":"course join after start date late enrollment eligibility"}', call_id='call_4MR5ZnRZCSFdHdDuNWqStJiB', name='search', type='function_call', id='fc_68c1be670aa881a09bdce4704849e89709140a3903ee0d2c', status='completed')
Yes, you can still join the course even after it has started! While official registration may close, you're eligible to engage with the materials and submit homework assignments without registering. Just keep in mind that there will be deadlines for the final projects, so try not to wait until the last minute to complete them.

Additionally, it may be helpful to familiarize yourself with the course materials and participate in any available office hours or discussions.

Would you like information o

## Using RAG chat agent class as a wrapper

In [16]:
#!pip install toyaikit --q

from toyaikit.llm import OpenAIClient
from toyaikit.tools import Tools
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import OpenAIResponsesRunner
from toyaikit.chat.runners import DisplayingRunnerCallback

In [17]:
# instantiate tools objects

agent_tools = Tools()
agent_tools.add_tool(search, search_tool)

In [18]:
# chat interface makes output prettier - 
# as it is actually creating markdown responses which look much better than plain text above 
chat_interface = IPythonChatInterface()

# runner object instantiates our while loop for chat agent like we did above
runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    llm_client=OpenAIClient()
)

In [19]:
callback = DisplayingRunnerCallback(chat_interface)
# to display clickable black triangles for debugging to see what is inside calls to LLM

In [20]:
messages = runner.loop(prompt='how do I install kafka', callback=callback) 
# enjoy better clickable markdown formatting below!

In [21]:
# playing again with RAG LLM call with our search tool use under the hood:
# - lets now have a follow up question -
# before LLm asked << Would you like to explore more about Kafka topics or configurations?  >>
# follow up question is 'I want to use docker'
# and we need to provide the history as llm is stateless - << previous_messages=messages >>
# thus we save the follow-up answer in a new variable new_messages to keep track of our conversation:

new_messages = runner.loop(
    prompt='I want to use docker',
    previous_messages=messages,
    callback=callback,
)

In [22]:
# finally full search agent implemented with interface
# type STOP to exit as usual

messages = runner.run();
# NB - in Jupyter, a trailing semicolon suppresses the automatic output display of the last expression.
# It’s basically a trick to keep notebooks tidy when you don’t want large outputs printed.

You: stop


Chat ended.


## RAG - LLM tool use - adding more tools

In [23]:
# now we will have 2 tools - search and add entry
# Alexey Grigoriev usually asks ChatGPT to add doc strings and type hints to functions
# thus tools can be created faster:

from typing import List, Dict, Any

def search(query: str) -> List[Dict[str, Any]]:
    """
    Search the FAQ database for entries matching the given query.

    Args:
        query (str): Search query text to look up in the course FAQ.

    Returns:
        List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.
    """
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

def add_entry(question: str, answer: str) -> None:
    """
    Add a new entry to the FAQ database.

    Args:
        question (str): The question to be added to the FAQ database.
        answer (str): The corresponding answer to the question.
    """
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)



In [24]:
# lets actually add it:

agent_tools = Tools()
agent_tools.add_tool(search)
agent_tools.add_tool(add_entry)

In [25]:
# and check that both tools are added:

agent_tools.get_tools()

[{'type': 'function',
  'name': 'search',
  'description': 'Search the FAQ database for entries matching the given query.\n\nArgs:\n    query (str): Search query text to look up in the course FAQ.\n\nReturns:\n    List[Dict[str, Any]]: A list of search result entries, each containing relevant metadata.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': 'query parameter'}},
   'required': ['query'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'add_entry',
  'description': 'Add a new entry to the FAQ database.\n\nArgs:\n    question (str): The question to be added to the FAQ database.\n    answer (str): The corresponding answer to the question.',
  'parameters': {'type': 'object',
   'properties': {'question': {'type': 'string',
     'description': 'question parameter'},
    'answer': {'type': 'string', 'description': 'answer parameter'}},
   'required': ['question', 'answer'],
   'additionalProperties': False}}]

In [26]:
# set up our agent loop wrapper 

runner = OpenAIResponsesRunner(
    tools=agent_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    llm_client=OpenAIClient()
)


In [27]:
# and run it - type STOP as usual to exit:

runner.run();
# when we get answer we now can ask LLM to add it back to FAQ
# I ask model What is the temperature of boiling water - this is NOT in our FAQ
# when it answers I will ask to add it to FAQ using our 'add_entry' tool:


You: What is the temperature of boiling water


You: Add this back to FAQ


You: stop


Chat ended.


In [28]:
# You: Add this back to FAQ
# Function call: add_entry({"question":"What is the temperature of boiling...)
# lets check if it is added there -

index.docs[-1]

# it prints:
# {'question': 'What is the temperature of boiling water?',
#  'text': 'The boiling point of water is typically 100 degrees Celsius (212 degrees Fahrenheit) at sea level. It can vary depending on atmospheric pressure; for example, at higher altitudes, water boils at a lower temperature due to reduced pressure.',
#  'section': 'user added',
#  'course': 'data-engineering-zoomcamp'}


# this happens because above we defined
# doc = {
#         'question': question,
#         'text': answer,
#         'section': 'user added',
#         'course': 'data-engineering-zoomcamp'
#     }

{'question': 'What is the temperature of boiling water?',
 'text': 'The boiling point of water is typically 100 degrees Celsius (212 degrees Fahrenheit) at sea level. It can vary depending on atmospheric pressure; for example, at higher altitudes, water boils at a lower temperature due to reduced pressure.',
 'section': 'user added',
 'course': 'data-engineering-zoomcamp'}